<a href="https://colab.research.google.com/github/mingusmingus/MONET/blob/main/Proyecto/MONET_v1_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#--------------------------------------------------------------------------------------#
# GUI AJUSTES MENORES: IMPLEMENTACIÓN DE DESCARGAS Y CAMBIO DE FORMATO .webp a .jpg    #
#--------------------------------------------------------------------------------------#


In [ ]:
# Instalar Hugging Face Transformers y Diffusers para generación de texto e imágenes
!pip install transformers diffusers accelerate torch torchvision

# Instalar TextBlob para detección de emociones
!pip install textblob

# Descargar datos de entrenamiento para TextBlob (necesario para análisis de texto)
!python -m textblob.download_corpora

# Instalar deep-translator y Gradio
!pip install deep-translator gradio

ERROR: Operation cancelled by user
Traceback (most recent call last):
object address  : 0x7c01faf36ec0
object refcount : 2
object type     : 0x9d5ea0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C
  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
  Using cached gradio-5.13.1-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.7-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.6.0-py3-none-any.whl.metadata (7.1 kB)
ERROR: Operation cancelled by user


In [ ]:
from transformers import pipeline
from deep_translator import GoogleTranslator
from diffusers import StableDiffusionPipeline
from PIL import Image  # Importar PIL para manipulación de imágenes
import time

# Cargar el modelo de clasificación de emociones
clasificador_emociones = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Cargar el modelo Stable Diffusion
modelo_sd = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
def detectar_emocion(texto):
    # Traducir el texto al inglés usando deep-translator
    texto_traducido = GoogleTranslator(source='auto', target='en').translate(texto)

    # Detectar emoción en inglés
    resultados = clasificador_emociones(texto_traducido)
    emocion_dominante = max(resultados[0], key=lambda x: x['score'])

    # Diccionario de traducción de emociones
    emociones_traducidas = {
        "joy": "alegría",
        "sadness": "tristeza",
        "anger": "enojo",
        "fear": "miedo",
        "love": "amor",
        "surprise": "sorpresa"
    }
    return emociones_traducidas.get(emocion_dominante['label'], "desconocida")

def generar_imagen(emocion):
    descripciones = {
        "alegría": "a bright and colorful painting of a sunny meadow with flowers",
        "tristeza": "a dark and moody oil painting of a rainy cityscape",
        "enojo": "an abstract painting with bold red and black strokes",
        "miedo": "a surreal painting of a shadowy forest with eerie lighting",
        "amor": "a romantic scene of a couple under a glowing sunset",
        "sorpresa": "a vibrant and dynamic painting with explosive colors and shapes"
    }

    descripcion = descripciones.get(emocion, "a beautiful abstract art piece")
    imagen = modelo_sd(descripcion).images[0]

    # Convertir la imagen a RGB para guardarla como JPG
    imagen = imagen.convert("RGB")

    return imagen

In [ ]:
import gradio as gr

def guardar_imagen(imagen, emocion_detectada):
    """
    Guarda la imagen generada en el entorno de Colab y crea un enlace de descarga.
    """
    # Crear un nombre de archivo basado en el timestamp
    timestamp = time.strftime("%Y%m%d-%H%M%S")  # Genera la fecha y hora actual en formato único
    nombre_imagen = f"{emocion_detectada}_{timestamp}.jpg"  # Cambiar a .jpg
    ruta_guardado_colab = f"/content/{nombre_imagen}"

    # Guardar la imagen en formato JPG
    imagen.save(ruta_guardado_colab, format="JPEG")  # Guardar como JPG
    print(f"Imagen guardada en: {ruta_guardado_colab}")

    return ruta_guardado_colab, nombre_imagen

def procesar_texto(sentimiento_usuario):
    emocion_detectada = detectar_emocion(sentimiento_usuario)
    imagen_generada = generar_imagen(emocion_detectada)

    # Guardar imagen y obtener la ruta
    ruta_imagen, nombre_imagen = guardar_imagen(imagen_generada, emocion_detectada)

    return ruta_imagen, emocion_detectada

# Crear la interfaz Gradio con la nueva sintaxis
iface = gr.Interface(
    fn=procesar_texto,
    inputs=gr.Textbox(label="¿Cómo te sientes hoy?"),
    outputs=[gr.Image(label="Imagen Generada"), gr.Textbox(label="Emoción Detectada")],
    title="Detección de Emociones y Generación de Imágenes",
    description="Introduce tu sentimiento y genera una imagen basada en la emoción detectada."
)

# Ejecutar la interfaz :)
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d639d9e598703c0e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
